In [12]:
import os.path as op
import os
import matplotlib.pyplot as plt
import shutil
import json
import numpy as np
import pandas as pd

from fpdf import FPDF
from mne.preprocessing import read_ica
import mne_bids
import sys

import mne
import matplotlib.pyplot as plt
import mne_bids

In [2]:
subject_id="CB040"
visit_id="1"
bids_root="/headnode1/abry4213/data/Cogitate_MEG_challenge"
record="run"
has_eeg=False

In [3]:
# Prepare PDF report
pdf = FPDF(orientation="P", unit="mm", format="A4")

# Set path to preprocessing derivatives
prep_deriv_root = op.join(bids_root, "derivatives", "preprocessing")
prep_figure_root =  op.join(prep_deriv_root,
                            f"sub-{subject_id}",f"ses-{visit_id}","meg",
                            "figures")
prep_report_root =  op.join(prep_deriv_root,
                            f"sub-{subject_id}",f"ses-{visit_id}","meg",
                            "reports")
prep_code_root = op.join(prep_deriv_root,
                            f"sub-{subject_id}",f"ses-{visit_id}","meg",
                            "codes")

print("Processing subject: %s" % subject_id)

Processing subject: CB040


In [5]:
print("Processing subject: %s" % subject_id)

data_path = os.path.join(bids_root,f"sub-{subject_id}",f"ses-{visit_id}","meg")

fname = bids_root + f"sub-{subject_id}/ses-1/meg/sub-{subject_id}_ses-{visit_id}_task-dur_run-01_meg.json"

if "run" in fname:
    run = fname.split("run-")[1].split("_")[0]
elif "rest" in fname:
    run = None
print("  Run: %s" % run)

# Set task
if 'dur' in fname:
    bids_task = 'dur'
elif 'vg' in fname:
    bids_task = 'vg'
elif 'replay' in fname:
    bids_task = 'replay'
elif "rest" in fname:
    bids_task = "rest"
else:
    raise ValueError("Error: could not find the task for %s" % fname)

Processing subject: CB040
  Run: 01


In [6]:
# Set BIDS path
bids_path_annot = mne_bids.BIDSPath(
    root=prep_deriv_root, 
    subject=subject_id,  
    datatype='meg',  
    task=bids_task,
    run=run,
    session=visit_id, 
    suffix='annot',
    extension='.fif',
    check=False)

# Read raw data
raw = mne_bids.read_raw_bids(bids_path_annot).load_data()

Opening raw data file /headnode1/abry4213/data/Cogitate_MEG_challenge/derivatives/preprocessing/sub-CB040/ses-1/meg/sub-CB040_ses-1_task-dur_run-01_annot.fif...


/tmp/ipykernel_29731/1032540291.py:14: RuntimeWarning: This filename (/headnode1/abry4213/data/Cogitate_MEG_challenge/derivatives/preprocessing/sub-CB040/ses-1/meg/sub-CB040_ses-1_task-dur_run-01_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne_bids.read_raw_bids(bids_path_annot).load_data()


    Range : 10000 ... 848999 =     10.000 ...   848.999 secs
Ready.
Reading 0 ... 838999  =      0.000 ...   838.999 secs...


/tmp/ipykernel_29731/1032540291.py:14: RuntimeWarning: Did not find any events.tsv associated with sub-CB040_ses-1_task-dur_run-01.

The search_str was "/headnode1/abry4213/data/Cogitate_MEG_challenge/derivatives/preprocessing/sub-CB040/**/meg/sub-CB040_ses-1*events.tsv"
  raw = mne_bids.read_raw_bids(bids_path_annot).load_data()
/tmp/ipykernel_29731/1032540291.py:14: RuntimeWarning: Did not find any channels.tsv associated with sub-CB040_ses-1_task-dur_run-01.

The search_str was "/headnode1/abry4213/data/Cogitate_MEG_challenge/derivatives/preprocessing/sub-CB040/**/meg/sub-CB040_ses-1*channels.tsv"
  raw = mne_bids.read_raw_bids(bids_path_annot).load_data()
/tmp/ipykernel_29731/1032540291.py:14: RuntimeWarning: Did not find any meg.json associated with sub-CB040_ses-1_task-dur_run-01.

The search_str was "/headnode1/abry4213/data/Cogitate_MEG_challenge/derivatives/preprocessing/sub-CB040/**/meg/sub-CB040_ses-1*meg.json"
  raw = mne_bids.read_raw_bids(bids_path_annot).load_data()
/tmp

In [11]:
# Find response events
response = mne.find_events(raw,
                            stim_channel='STI101',
                            consecutive = False,
                            mask = 65280,
                            mask_type = 'not_and'
                        )
response = response[response[:,2] == 255]

# Find all other events
events = mne.find_events(raw,
                            stim_channel='STI101',
                            consecutive = True,
                            min_duration=0.001001,
                            mask = 65280,
                            mask_type = 'not_and'
                        )
events = events[events[:,2] != 255]

# Concatenate all events
events = np.concatenate([response,events],axis = 0)
events = events[events[:,0].argsort(),:]

2067 events found on stim channel STI101
Event IDs: [  1   2   3   4   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  86  96  97 101 102 103 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 151 152 153 161
 162 163 164 165 166 167 168 201 202 203 255]
2051 events found on stim channel STI101
Event IDs: [  1   2   3   4   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  86  96  97 101 102 10

In [20]:
eve = events.copy()
events = eve[eve[:, 2] < 81].copy()
metadata = {}
metadata = pd.DataFrame(metadata, index=np.arange(len(events)),
                        columns=['Stim_trigger', 'Category',
                                    'Orientation', 'Duration',
                                    'Task_relevance', 'Trial_ID',
                                    'Response', 'Response_time(s)'])
Category = ['face', 'object', 'letter', 'false']
Orientation = ['Center', 'Left', 'Right']
Duration = ['500ms', '1000ms', '1500ms']
Relevance = ['Relevant target', 'Relevant non-target', 'Irrelevant']
k = 0

for i in range(eve.shape[0]):
    if eve[i, 2] < 81:
    ##find the end of each trial (trigger 97)
        time = [t for t, j in enumerate(eve[i:i + 9, 2]) if j == 97][0]
        metadata.loc[k]['Stim_trigger'] = eve[i,2]
        metadata.loc[k]['Category'] = Category[int((eve[i,2]-1)//20)]
        metadata.loc[k]['Orientation'] = Orientation[[j-100 for j in eve[i:i+time,2]
                                                        if j in [101,102,103]][0]-1]
        metadata.loc[k]['Duration'] = Duration[[j-150 for j in eve[i:i+time,2]
                                                if j in [151,152,153]][0]-1]
        metadata.loc[k]['Task_relevance'] = Relevance[[j-200 for j in eve[i:i+time,2]
                                                        if j in [201,202,203]][0]-1]
        metadata.loc[k]['Trial_ID'] = [j for j in eve[i:i+time,2]
                                        if (j>110) and (j<149)][0]
        metadata.loc[k]['Response'] = True if any(eve[i:i+time,2] == 255) else False
        if metadata.loc[k]['Response'] == True:
            r = [r for r,j in enumerate(eve[i:i+time,2]) if j == 255][0]
            metadata.loc[k]['Response_time(s)'] = (eve[i+r,0] - eve[i,0])
        # miniblock = [j for j in eve[i:i+t,2] if (j>160) and (j<201)]
        # metadata.loc[k]['Miniblock_ID'] = miniblock[0] if miniblock != [] else np.nan
        k += 1


IndexError: list index out of range

In [25]:

[j for t, j in enumerate(eve[i:i + 9, 2])]

[74, 39, 20, 47, 42, 74, 20, 74, 41]